In [1]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral7
from bokeh.models.tools import HoverTool

In [2]:
output_notebook()

Loading BokehJS ...

## Loans by date

### Prepare data

In [22]:
df = pd.read_csv('alma_loans_by_date.csv')  # read data to dataframe
df['Loan Date'] = pd.to_datetime(df['Loan Date'])  # set column to datetime format
df.set_index('Loan Date', inplace=True)  #  set index to 'Loan Date' in order to group by month
g = df.groupby(pd.Grouper(freq="M"))  # group by month
new_df = g.sum()  # sum groupby object
new_df.reset_index(inplace=True)  # reset index 

### Loans by month

In [19]:
source = ColumnDataSource(new_df)
# create the 'canvas' for the visualization
p = figure(plot_height=500, plot_width=950, x_axis_type='datetime')

# create line glyph
p.line(x='Loan Date', y='Loans (In House + Not In House)', line_width=2, source=source, color=Spectral7[0],
      legend='Total Loans')

# add metadata to the visualization
p.yaxis.axis_label = 'Total Loans'
p.xaxis.axis_label = 'Month'
p.title.text = 'Iliff Library Loans by Month'
p.legend.location = 'top_left'

# display visualization
show(p)

### Loans by day

In [24]:
source = ColumnDataSource(df)
p = figure(plot_height=500, plot_width=950, x_axis_type='datetime')
p.line(x='Loan Date', y='Loans (In House + Not In House)', line_width=2, source=source, color=Spectral7[0],
      legend='Number of Loans')

# add metadata to the visualization
p.yaxis.axis_label = 'Total Loans'
p.xaxis.axis_label = 'Day'
p.title.text = 'Iliff Library Loans by Day'
p.legend.location = 'top_left'

# display visualization
show(p)

## Loans by item

In [25]:
items_df = pd.read_csv('alma_loans_by_item.csv')

In [34]:
items_df['num_loans'] = pd.cut(items_df['num_loans'], bins=[-1, 0, 1, 3, 6, 9, 118],
       labels=['0', '1', '2-3', '4-6', '7-9', '10+'])

In [40]:
new_df = items_df.groupby('num_loans').count()

In [42]:
new_df.reset_index(inplace=True)

In [43]:
new_df.head()

,num_loans,Barcode
0,0,89158
1,1,23627
2,2-3,19974
3,4-6,11343
4,7-9,4592


In [50]:
source = ColumnDataSource(new_df)
groupings = source.data['num_loans'].tolist()
p = figure(plot_width=700, plot_height=500, x_range=groupings)

p.title.text = 'Number of Items with Loans'
p.xaxis.axis_label = 'Number of Loans'
p.yaxis.axis_label = 'Number of Items'

p.vbar(x='num_loans', top='Barcode', source=source, width=0.2)
show(p)

## 10 Most popular LC categories

In [51]:
lc_df = pd.read_csv('loans_by_LC.csv')

In [54]:
new_df = lc_df.sort_values(by='num_loans', ascending=False).head(10)

In [55]:
new_df

,Classification Code,num_items,num_loans
18,BS,20144,69728
19,BT,11068,29278
20,BV,15108,28290
21,BX,20615,22817
17,BR,11689,22580
13,BL,6479,14800
7,B,4106,6841
14,BM,2569,6171
50,E,1959,3452
10,BF,2389,3432
